<hr style="height:2px;">

# Demo: Training data generation for isotropic reconstruction of Zebrafish retina

### Notes

- Documentation available: http://csbdeep.bioimagecomputing.com/doc/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
from tifffile import imread
from csbdeep import datagen
from csbdeep.plot_utils import plot_some

<hr style="height:2px;">

# Download example data

In [ ]:
from csbdeep.utils import download_and_extract_zip_file
download_and_extract_zip_file(
    url = 'https://cloud.mpi-cbg.de/index.php/s/Vu0rN1G33z9hQa4/download',
    provides = ('raw_data/retina/cropped_farred_RFP_GFP_2109175_2color_sub_10.20.tif',)
)

Data should have been downloaded like this:

    raw_data/
    └── retina
        └── cropped_farred_RFP_GFP_2109175_2color_sub_10.20.tif

<hr style="height:2px;">

# Raw 3D image stack with low z resolution

We only show one stack for demonstration, but you should have different stacks at different developmental timepoints.

In [ ]:
x = imread('raw_data/retina/cropped_farred_RFP_GFP_2109175_2color_sub_10.20.tif')
axes = 'ZCYX'
subsample = 10.2
print('image size       =', x.shape)
print('image axes       =', axes)
print('subsample factor =', subsample)


plt.figure(figsize=(15,15))
plot_some(np.moveaxis(x,1,-1)[[5,-5]], title_list=[['xy slice','xy slice']], pmin=2,pmax=99.8);

plt.figure(figsize=(15,15))
plot_some(np.moveaxis(np.moveaxis(x,1,-1)[:,[50,-50]],1,0), title_list=[['xz slice','xz slice']], pmin=2,pmax=99.8, aspect=subsample);

<hr style="height:2px;">

# Dev

In [ ]:
#from csbdeep.plot_utils import to_color
#raw_data = datagen.get_tiff_pairs_from_folders (
#    basepath   = 'raw_data',
#    source_dirs = ['retina'],
#    target_dir = 'retina',
#    axes = axes,
#)
#iso_transform = datagen.anisotropic_distortions(
#    psf = None,
#    #psf = np.ones((1,1,1,1)),
#    #psf_axes = 'XYCZ',
#    poisson_noise=0,
#    gauss_sigma=0,
#    subsample = 5,
#)
#
#gen_iso = iso_transform.generator(raw_data.generator())
#print(iso_transform.name)
#for x,y,axes,mask in gen_iso:
#    plt.figure(figsize=(16,6))
#    print(x.shape, y.shape)
#    a,b = x[10,1],y[10,1]
#    c = to_color(np.stack([a,b]))
#    plt.subplot(131); plt.imshow(a)
#    plt.subplot(132); plt.imshow(b)
#    plt.subplot(133); plt.imshow(c)
#None;

<hr style="height:2px;">

# Patch generation

From corresponding stacks, we generate 3D patches of size (1,2,128,128), where we drop the z dimensions before saving the patches to disk

In [ ]:
raw_data = datagen.get_tiff_pairs_from_folders (
    basepath    = 'raw_data',
    source_dirs = ['retina'],
    target_dir  = 'retina',
    axes        = axes,
)

iso_transform = datagen.anisotropic_distortions(
    subsample = 10.2,
    psf       = None,
)

X, Y, XY_axes = datagen.create_patches (
    raw_data            = raw_data,
    patch_size          = (1,2,128,128),
    #patch_axes          = axes,
    n_patches_per_image = 512,
    transforms          = [iso_transform],
)

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)

In [ ]:
# remove dummy z dim to obtain multi-channel 2D patches
X = X[:,:,0,...]
Y = Y[:,:,0,...]
XY_axes = XY_axes.replace('Z','')

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)

## Save extracted training data to disk

We will load this later to train the neural network.

In [ ]:
np.savez('my_training_data.npz', X=X, Y=Y, axes=XY_axes)

## Show

Show some of those patch pairs (first row: *source*, second row: *target*)

In [ ]:
from csbdeep.plot_utils import plot_some

for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1))
    plot_some(np.moveaxis(X[sl],1,-1),np.moveaxis(Y[sl],1,-1),title_list=[np.arange(sl.start,sl.stop)])
    plt.show()
None;